<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install chardet
%pip install ftfy

In [1]:
from openai import OpenAI
import os
import uuid
import json
from ftfy import fix_encoding

client = OpenAI()

In [2]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    return encoding


In [ ]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)

print(f'Books in list: {len(filenames)}')

In [5]:
#generate quetsions to the "answers" extracted from the text
def answer_gpt(answer):
    response = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            { 
                "role": "system", 
                "content": "You are assisting in the generation of training data for fine-tuning. You will receive a chunk of text, and will respond with a short casually phrased question to which the chunk of text you received would be an expected answer. The person who generated the answer is Alan Watt's and often he will give a response that answers a question only indirectly. The question should not exactly contain the subject matter of the answer. The question you create should be one to which the answer would be a correct indirect or metaphorical answer." 
            },
            {
                "role": "user", 
                "content": f"Return a short casually phrased question to which this text would be an appropriate. The question should not reference the core subject matter of the answer in an overt way. : {answer}"
            }
        ]
    )
    return response.choices[0].message.content.strip()

In [4]:
#split chunks into sections that could reasonably be the answer to a question
def chunk_gpt(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "
            },
            {
                "role": "user", 
                "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"
            }
        ]
    )
    return response.choices[0].message.content.strip().split('\n') 

In [7]:
#loop through all of the processed "answers" and generate questions
def process_questions(processed_answers, book_name):
    for i in range(len(processed_answers)):
        
        #limiting cycles for testing
        if i > 2 and debugger == True:
            continue

        answer = processed_answers[i]
        #remove blanks
        if len(answer) < 30:
            continue

        #question = answer_gpt(answer)

        json_obj = { 
            "book" : book_name,
            "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.", 
            "input" : "", 
            "output" : answer 
        }
        
        processed_json.append(json_obj)

In [8]:
#loop through the chunks of a book
def process_book_chunks(text_chunks, book_name):
    print(f'Processing {len(text_chunks)} chunks for {book_name}')
    for i in range(len(text_chunks)):
        chunk = fix_encoding(text_chunks[i])
        
        #skip the first and last pages which are usually credits and other misellaneous content
        if i > len(text_chunks)-8 or i < 8:
            continue

        #limiting requests for testing purposes
        if i > 9 and debugger == True:
            continue

        chunk_entry(chunk, book_name)
        #processed_answers = chunk_gpt(chunk)
        #process_questions(processed_answers, book_name)
    print(f'Completed processing {book_name}')

In [9]:
#process books into chunks of characters
def process_books():
    chunk_size = 5000 
    for i in range(len(filenames)):
        
        #limiting cycles for testing
        if i > 0 and debugger == True:
            continue
        
        encoding = detect_encoding(f'books/{filenames[i]}')
        with open(f'books/{filenames[i]}', 'r', encoding=encoding, errors='replace') as file:
            content = file.read().replace('\n','')
            text_chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]

            process_book_chunks(text_chunks, filenames[i])
            

    #print(fix_encoding(json.dumps(processed_json, indent=4, ensure_ascii=False)))
    print(fix_encoding(json.dumps(chunk_list, indent=4, ensure_ascii=False)))

In [10]:
chunk_list = []
def chunk_entry(chunk, book):
    bulk_entry = { "custom_id": f"{book}-{str(uuid.uuid4())}", "method": "POST", "url": "/v1/chat/completions", "body": { "model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed."},{"role": "user", "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{chunk}"}],"max_tokens": 5000}}
    chunk_list.append(bulk_entry)

In [13]:
processed_json = [] 

In [43]:
file_path = 'batch_answer_upload.jsonl'

with open(file_path, 'w', encoding='utf-8') as file:
    for entry in chunk_list:
        file.write(json.dumps(entry, ensure_ascii=False) + '\n')

In [10]:
batch_input_file = client.files.create(
  file=open("batch_answer_upload.jsonl", "rb"),
  purpose="batch"
)

In [ ]:
%pip install --upgrade openai --quiet

In [ ]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "processing chunks into answers"
    }
)

In [ ]:
client.batches.retrieve("batch_eoJiEIRMqNH9P5oKCu7LO6M9")

In [ ]:
debugger = False
process_books()

In [31]:
with open('chunks_3_through_189','r') as f1:
    book_one = json.load(f1)

with open('more_chunks', 'r') as f2:
    book_two = json.load(f2)

combined_data = book_one + book_two

with open('first_batch_total', 'w') as f_combined:
    json.dump(combined_data, f_combined, indent=4)

In [ ]:
print(fix_encoding(json.dumps(processed_json, indent=4, ensure_ascii=False)))